In [2]:
from ib_insync import *
import pandas as pd
import random
from datetime import date, time

In [3]:
# get the list of tickers
ticker_df = (pd.read_excel("data/isin_list.xlsx")
                .loc[lambda x: ~x["isin"].str.startswith("_") & (x["isin"] != "Unassigned")]
                .drop_duplicates()
                .dropna()
                .sort_values("isin"))
ticker_df

,isin
4756,AEA000201011
4804,AEA000801018
14631,AEA001901015
5389,AEA002001013
9377,AEA002401015
...,...
4815,ZAE000298253
4253,ZAE000314084
13887,ZAE000332789
13751,ZAE000339891


In [4]:


# 1. Setup
util.startLoop() 
ib = IB()

# 2. Connect
my_client_id = random.randint(1, 9999)
try:
    ib.connect('127.0.0.1', 7497, clientId=my_client_id)
    print("✅ Connected!")
except Exception as e:
    print(f"❌ Error: {e}")

# 3. Market Data Type (3 = Delayed)
ib.reqMarketDataType(3)


# scratch the data
price_data_ls = []
isin_list = ticker_df["isin"].unique()
for idx, isin in enumerate(isin_list):

    # resolve ISIN
    contract = Contract(
        secType='STK',
        secIdType='ISIN',
        secId=isin,
        exchange='SMART'
    )
    details = ib.reqContractDetails(contract)
    contract_ls = [contract for contract in details if contract.contract.primaryExchange == "IBIS"]
    if len(contract_ls) == 0:
        continue
    contract = contract_ls[0].contract
    symbol_local = contract.localSymbol
    symbol = contract.symbol
    conId = contract.conId

    # get the ticker data
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='10 Y',
        barSizeSetting='1 day',
        whatToShow='TRADES',
        useRTH=True
    )

    # pandas Processing
    df = util.df(bars)
    if df is not None and not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename({"date": "datetime"}, axis=1)
        df["date"] = df["datetime"].dt.date
        df["time"] = df["datetime"].dt.time
        df["hour"] = df["datetime"].dt.hour
        df["symbol_local"] = symbol_local
        df["symbol"] = symbol
        df["isin"] = isin
        df["contract"] = conId
        price_data_ls.append(df)
        print(f"SUCCESS - {round(100*idx/len(isin_list), 2)}% - {symbol}")
    else:
        print("⚠️ No data received. Try changing 'whatToShow' to 'MIDPOINT' or check market hours.")

# disconnect
ib.disconnect()

# put data together
price_data_df = pd.concat(price_data_ls)
price_data_df.to_csv("data/stocks.csv", index=False)
price_data_df

Error 200, reqId 3: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA000201011')


✅ Connected!


Error 200, reqId 4: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA000801018')
Error 200, reqId 5: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA001901015')
Error 200, reqId 6: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA002001013')
Error 200, reqId 7: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA002401015')
Error 200, reqId 8: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA003001012')
Error 200, reqId 9: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='AEA004601018'

SUCCESS - 0.82% - KTN
SUCCESS - 3.34% - VUL


Error 200, reqId 353: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='BMG2113M1203')
Error 200, reqId 354: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='BMG2164J1099')
Error 200, reqId 367: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='BMG4404N1149')
Error 200, reqId 370: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='BMG4600H1198')
Error 200, reqId 374: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='BMG524181036')
Error 200, reqId 375: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='B

...

SUCCESS - 97.84% - 3V64
SUCCESS - 98.08% - WB2
SUCCESS - 98.09% - WMT


Error 200, reqId 7756: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='US931CVR0133')


SUCCESS - 98.14% - J5A
SUCCESS - 98.34% - NWT
SUCCESS - 98.46% - WDC
SUCCESS - 98.58% - WMB
SUCCESS - 98.83% - WYR


Error 200, reqId 7847: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='VGG2R04S1048')
Error 200, reqId 7849: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='VGG3165E1079')
Error 200, reqId 7853: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='ZAE000005443')
Error 200, reqId 7855: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='ZAE000009932')
Error 200, reqId 7861: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', secId='ZAE000026480')
Error 200, reqId 7862: No security definition has been found for the request, contract: Contract(secType='STK', exchange='SMART', secIdType='ISIN', se

,datetime,open,high,low,close,volume,average,barCount,date,time,hour,symbol_local,symbol,isin,contract
0,2016-02-03,5.376,5.450,5.330,5.404,67956.0,5.3644,143,2016-02-03,00:00:00,0,KTN,KTN,AT0000A0E9W5,62078268
1,2016-02-04,5.454,5.482,5.302,5.460,78552.0,5.3974,161,2016-02-04,00:00:00,0,KTN,KTN,AT0000A0E9W5,62078268
2,2016-02-05,5.536,5.580,5.452,5.520,75697.0,5.5186,150,2016-02-05,00:00:00,0,KTN,KTN,AT0000A0E9W5,62078268
3,2016-02-08,5.450,5.526,5.130,5.130,149680.0,5.2740,185,2016-02-08,00:00:00,0,KTN,KTN,AT0000A0E9W5,62078268
4,2016-02-09,5.082,5.112,4.560,4.800,356940.0,4.8276,509,2016-02-09,00:00:00,0,KTN,KTN,AT0000A0E9W5,62078268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,2026-01-23,98.320,98.320,96.370,96.370,100.0,96.3895,6,2026-01-23,00:00:00,0,WYR,WYR,US9831341071,79891335
2241,2026-01-26,96.480,96.480,96.480,96.480,68.0,96.4800,1,2026-01-26,00:00:00,0,WYR,WYR,US9831341071,79891335
2242,2026-01-27,95.920,95.920,95.320,95.320,43.0,95.9035,3,2026-01-27,00:00:00,0,WYR,WYR,US9831341071,79891335
2243,2026-01-29,91.340,91.340,91.340,91.340,1.0,91.3400,1,2026-01-29,00:00:00,0,WYR,WYR,US9831341071,79891335
